In [1]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import struct
import time
#import csv


# Function to extract k bits from p position and returns the extracted value as integer 
def bitExtracted(number, k, p):    
    return ( ((1 << k) - 1)  &  (number >> (p) ) ); 
     
# Get Data out of Binary File
def readdata(filename,nEvents):
    df_energies = pd.DataFrame()
    df_traces = pd.DataFrame()
    #Read data from a file
    with open(filename, "rb") as file:
            #initialise arrays used in the loop
            energyraw = [[] for j in range(16)]
            traceraw = [[] for j in range(nEvents)]
            #print (energyraw)
        
            #loop over the events
            for i in range(nEvents): 
                #reading firstword of header that contains info on the event
                nbyte_firstword=4
                header_firstword = file.read(nbyte_firstword)
                header_firstword_int=int.from_bytes(header_firstword, byteorder = "little")
                #header_firstword_string = '{0:32b}'.format(header_firstword_int)
            
                # use the function bitExtract to extract the information contained in the first line as per Pixie16 manual
                channelno=bitExtracted(header_firstword_int, 4, 0)
                slotid=bitExtracted(header_firstword_int, 4, 4)
                crateid=bitExtracted(header_firstword_int, 4, 8)
                headerlgth=bitExtracted(header_firstword_int, 5, 12)
                eventlgth=bitExtracted(header_firstword_int, 14, 17)
                finishcode=bitExtracted(header_firstword_int, 1, 31)
        
              # channelno_string = '{0:04b}'.format(channelno)
              # slotid_string = '{0:04b}'.format(slotid)
              #  crateid_string = '{0:04b}'.format(crateid)
              #  headerlgth_string = '{0:05b}'.format(headerlgth)
              #  eventlgth_string = '{0:14b}'.format(eventlgth)
              #  finishcode_string = '{0:01b}'.format(finishcode)       
        
                #read the rest of the header to find the energyraw and tracelength values
                headerlgth_minus1=headerlgth-1
                header = file.read(4*(headerlgth_minus1))
                values = struct.unpack(f'={headerlgth_minus1}L', header)
        
                energyraw_value=bitExtracted(values[3-1], 16, 0)
              #  energyraw_value_string = '{0:04b}'.format(energyraw_value)
                

                df_energies = df_energies.append(pd.DataFrame({'event_id':[i],
                                                               'channel_id':[channelno],
                                                               'energyraw_value':energyraw_value}, 
                                                                columns=['event_id','channel_id','energyraw_value']))
        
                #energyraw[channelno].append(energyraw_value)
        
                tracelgth=bitExtracted(values[3-1], 15, 16)
                traceoutofrange=bitExtracted(values[3-1], 1, 31)
        
                #read the trace values according to the tracelength above
                trace = file.read(2*tracelgth)
                trace_values = struct.unpack(f'={tracelgth}H', trace)
            
                #traceraw[i]=trace_values
                sampling_values=np.arange(tracelgth)
                event_id=np.zeros(tracelgth)
                event_id[:]=i
                

                df_traces = df_traces.append(pd.DataFrame({'event_id':event_id, 
                                                              'sampling':sampling_values,
                                                              'trace_values':trace_values}, 
                                                                columns = ['event_id','sampling','trace_values']))
                
                #print(i, channelno, slotid, headerlgth, eventlgth, energyraw_value, tracelgth, traceoutofrange)  
                
    return df_energies, df_traces  
       
df_energies, df_traces = readdata('/Users/lunapellegri/Data/digiDAQ-LaBr/rawdata_mod0-DDAS-2020-11-23-12:04:57.bin',100)

#print(df_energies)

In [ ]:
# plot traces and energy obtained from data binary file
%matplotlib widget
#%matplotlib inline

plt.figure(1)
plt.xlabel('Sampling')
plt.ylabel('Amplitude [a.u.]')
plt.title('LaBr Digital DAQ traces')
plt.plot(df_traces['trace_values']) 
#plt.show()

plt.figure(2)
plt.xlabel('Energy Raw [a.u.]')
plt.ylabel('Counts [a.u.]')
plt.title('LaBr Digital DAQ Energy Raw')
df_energies['energyraw_value'].plot.hist(bins=100, legend=False) 
plt.show()


In [8]:
#subtract the baseline to the traces

def trace_baseline(group):
    return group.sort_values(by='sampling')[30:300]

df_traces_grouped = df_traces.groupby(['event_id'])
baseline_values = df_traces_grouped.apply(trace_baseline)

baseline_values = baseline_values.reset_index(drop=True)
baseline = pd.DataFrame()
baseline['baseline_values'] = baseline_values.groupby(['event_id'])['trace_values'].mean()

def subtract_baseline(group1, group2):
    grouped1 = group1.groupby(['event_id'])
    grouped2 = group2.groupby(['event_id'])
    df_traces_zerobaseline = grouped1['trace_values'].sub(grouped2['baseline_values'])
    return df_traces_zerobaseline

df_traces_zerobaseline = pd.DataFrame()
df_traces_zerobaseline

#df_traces_zerobaseline
        
#plot the traces with baseline at zero
#plt.plot(df_traces_zerobaseline) 
#plt.xlabel('Sampling')
#plt.ylabel('Amplitude [a.u.]')
#plt.title('LaBr Digital DAQ traces - zero baseline')
#plt.show()


""
